In [1]:
import music21 as m21
import re
from collections import Counter
from swalign_local import *

In [23]:
# Step 1: Read input data:
import music21 as m21
import re
from collections import Counter
from swalign_local import *
import csv
import glob

#score_path = "jkupddtest/bachmono/wtc2f20.krn"
score_path = "jkupddtest/beethovenmono/sonata01-3.krn"
#score_path = "jkupddtest/mozartmono/sonata04-2.krn"
#score_path = "jkupddtest/gibbonsmono/silverswan.krn"
#score_path = "jkupddtest/chopinmono/mazurka24-4.krn"
#score_path = '~/facets-search-engine/data/Beethoven9thOdeToJoy.xml'

raw_score = m21.converter.parse(score_path)
num_of_parts = len(raw_score.parts)

In [24]:
num_of_parts

2

In [25]:
voices = []
for part in raw_score.parts:
    voices.append(part)

In [26]:
# Step 2: Get diatonic scale degree sequence from all voices

def get_diatonic_intervals(voice):
    
    # get key / root information:
    roots = [key.tonic for key in voice.recurse().getElementsByClass(m21.key.Key)]
    diatonic_root = roots[0].diatonicNoteNum if roots != [] else voice.analyze('key').tonic.diatonicNoteNum
    #print(diatonic_root)
    key = voice.analyze('key')

    noteoffset = []
    midi = []
    #timesig = m21_score.getContextByClass(m21.meter.TimeSignature)
    #barlength = timesig.barduration

    dict_wordtonum = {"Unison": '0', "Second": '1', "Third": '2', "Fourth": '3', "Fifth": '4', "Sixth": '5', "Seventh": '6'}
        
    dict_encode_dia_intervals = {"6D": 'A', '5D': 'B', '4D': 'C', '3D':'D', '2D':'E', '1D':'F', 
                                     '1A':'G', '2A':'H', '3A':'I', '4A':'J', '5A': 'K', '6A':'L', '0A': 'M'}
        
    diatonic_intervals = []
    beatstrength = []
    previous_note = None

    onset = 0
    for thisnote in voice.recurse().notes:
        onset += thisnote.offset
        
        # We ignore rests
        if thisnote.isRest: 
                # If the rest is a full measure, part of a multi-measure rest: we need to adjust
                continue
        
        noteoffset.append(onset)
        beatstrength.append(thisnote.beatStrength)
        #if thisnote.isNote:
        #    midi.append(thisnote.pitch.midi)
        #elif thisnote.isChord:
        #    midi.append(thisnote.root().midi)
            
        if previous_note is None:
                previous_note = thisnote
        else:
                if thisnote.isNote and previous_note.isNote:
                    # gap = number of semi-tones of the current interval 
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isNote:
                    gap = thisnote.root().diatonicNoteNum - previous_note.pitch.diatonicNoteNum
                elif thisnote.isChord and previous_note.isChord:
                    gap = thisnote.root().diatonicNoteNum - previous_note.root().diatonicNoteNum
                elif thisnote.isNote and previous_note.isChord:
                    gap = thisnote.pitch.diatonicNoteNum - previous_note.root().diatonicNoteNum
                # if a pitch change is detected
                    
                if gap != 0:
                    
                    if gap > 0:
                        #  if the semi-tone difference between the current and the previous item > 0, it is an ascending interval.
                        direction = 'A'
                    else:
                        #  otherwise, it is a descending interval.
                        direction = 'D'

                    # Get intervals using music21
                    """
                            "directedSimpleNiceName" examples: "Descending Doubly-Diminished Fifth", "Ascending Perfect Fourth", "Ascending Doubly-Augmented Fourth"
                            "simpleName" examples: dd5, P5, AA4. There's no direction information
                            Since it only executes when a pitch interval is detected, "unison" refers to an augmented unison, a.k.a minor second
                    """
                    # take intervals between root notes if there exists any chord
                    if previous_note.isChord:
                        startnote = previous_note.root()
                    else:
                        startnote = previous_note
                    if thisnote.isChord:
                        endnote = thisnote.root()
                    else:
                        endnote = thisnote
                        
                    m21_interval_directed = m21.interval.Interval(noteStart=startnote, noteEnd=endnote).directedSimpleNiceName

                    arr_diatonic = m21_interval_directed.split(" ")

                    m21_generic = dict_wordtonum[arr_diatonic[-1]]
                    
                    # m21_interval: 2A, 3D etc...
                    m21_interval = m21_generic+direction
                    if m21_generic == "0":
                        m21_interval = '0A'
                    # to make each m21_interval unique, show direction and each as single character in string, we encode the diatonic intervals as letters
                    encode_interval = dict_encode_dia_intervals[m21_interval]
                    diatonic_intervals.append(encode_interval)

                else:
                    # We take the interval between two consecutive pitches as 0A
                    encode_interval = dict_encode_dia_intervals['0A']
                    diatonic_intervals.append(encode_interval)
                previous_note = thisnote
            
    string = ""
    for i in diatonic_intervals:
        string += str(i)
        
    return string, key, noteoffset, midi, beatstrength

In [27]:
strings = []
noteoffsets = []
midis = []
keys = []
beatstrengths = []

for voice in voices:
    string, key, noteoffset, midi, beatstrength = get_diatonic_intervals(voice)
    strings.append(string)
    noteoffsets.append(noteoffset)
    beatstrengths.append(beatstrength)
    midis.append(midi)
    keys.append(key)
    print(len(string), len(beatstrength))#, len(midi))
    print(string)
    print(len(noteoffset))
    #print(midi)
    print(beatstrength)
    

677 678
GEGEMDIDLFHMGEGEMDIDLFHMIJMMBGGIFFIBHDGLFBHDGMGEGEMDIDLFHMGEGEMDIDLFHMIJMMBGGIFFIBHDGLFBHDGHGEGEJEGEMHDGKFCHDGHFFHFFHFFGGFFGFFFGFFFGGFHAGHGHHFGGEMFGGEMFGGEMFGGEMMHDGFFBHDGDCJGEGEJEGEMHDGKFCHDGHFFHFFHFFGGFFGFFFGFFFGGFHAGHGHHFGGEMFGGEMFGGEMFGGEMMHDGFFBHDGDCJKFGEGGGFGEGGGFGGFFFFGGFEHHIBFJHALFBJFCFHEEFHFMKFGEGGGFGEGGGFGGFFFFGGFEHHIBFJHALFBJFCFHEEFHFMHFGEGGGFGEGGGFGEGGGEHHIHBFMEGGGFGEGGHGGDFGEGGJFGEGGGFFGFFFGFFFGFFCIFGEGGGFGEGGGFGGFFFFGGFEHHIECFFHICDHFGEGGGFGEGGGFGEGGGEHHIHBFMEGGGFGEGGHGGDFGEGGJFGEGGGFFGFFFGFFFGFFCIFGEGGGFGEGGGFGGFFFFGGFEHHIECFFHICHGEGEMDIDLFHMGEGEMDIDLFHMIJMMBGGIFFIBHDGLFBHDGHGEGEJEGEMHDGKFCHDGHFFHFFHFFGGFFGFFFGFFFGGFHAGHGHHFGGEMFGGEMFGGEMFGGEMMHDGFFBHDGDC
678
[0.5, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.25, 1.0, 0.5, 0.5, 1.0, 1.0, 1.0, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 0.5, 0.5, 1.0, 1.0, 0.5, 1.0

In [28]:
# Smith-Waterman alignment between the two input sequences
def swm_alignment(string1, string2):
    match = 2
    mismatch = -2
    scoring = NucleotideScoringMatrix(match, mismatch)
    sw = LocalAlignment(scoring)#, gap_extension_penalty = -5) 
    
    alignment = sw.align(string1, string2)
    alignment_strings = alignment.dump()

    simi_score = alignment.matches / (alignment.mismatches + alignment.matches)
    
    return alignment_strings, simi_score


In [29]:
def filter_and_gather_patterns(allthepatterns, fullseq):
    pat = fullseq.split('-')
    print("num of pat in this seq:", len(pat))    
    for pattern in pat:
        # Get rid of the sequence shorter than 4; combine all alignment patterns into all pattern list
        if len(pattern) > 2:
            allthepatterns.append(pattern)
    return allthepatterns

In [30]:
# compare each pair of voices!
allthepatterns = []

for i in range(0, len(voices)-1):
    for j in range(i+1, len(voices)):
        alignment_strings, simi_score = swm_alignment(strings[i], strings[j])
        if simi_score < 0.25:
            print("ignoring voice pair:", i, j)
            # if two voices are very different, getting patterns from their alignment won't make sense
            continue
        # get the patterns from the first voice of this pair
        fullseq1 = alignment_strings[0]
        fullseq2 = alignment_strings[1]
        #print("len:", len(fullseq1), len(fullseq2))
        
        # what if we make sure to segment whenever there's not identical items
        newseq1 = fullseq1[0]
        newseq2 = fullseq2[0]
        for temp in range(1, len(fullseq1)):
            if fullseq1[temp] == '-' and fullseq1[temp-1] == '-':
                newseq2+='-'
            else:
                newseq2+=fullseq2[temp]
            if fullseq2[temp] == '-' and fullseq2[temp-1] == '-':
                newseq1+='-'
            else:
                newseq1+=fullseq1[temp]
        
        allthepatterns = filter_and_gather_patterns(allthepatterns, newseq1)
        allthepatterns = filter_and_gather_patterns(allthepatterns, newseq2)
        

Query:  66 HFFGGFFGFFFGFFFGGFHAGMKG-----EM--GEMF--HMFHKFMM--G---HMAMLG----------AM---KEBBIDIGH-FHEHFHMBIHBIDIDIHFFGGFFGFFFGFFFGGFHAGMKG-----EM--GEMF--HMFHKFMMGHMAMLGAMKEBBIDIMHHGCFF-H--ICHHEFGEGGGFGEGGGFGEG----GGDGEICGMIMIHHGCFFH-----IC-HHEF-----GEGGGFGEGGGFGEG----GGDGE--ICGMIMMMLALAKB----H--F-----GEGGGFGEGGGFGEGGG------FGEGGGFGEGG--G-FGEGGGFFG---FFFGFFFGFFFCHHGCFFHICHHEFGEGGGFGEGGGF-GFFFFMJMLALAKBH----F------GEGGGFGEGGGFGEGGG------FGEGGGFGEGG--G-FGEGGGFFG---FFFGFFFGFFFCHHGCFFHICHHEFGEGGGFGEGGGF-GFFFFMMMGFFGBMGFFGH---FF---FG-G---------G----BI---H-----B--I-F--H---F-H----E-------H---F-HMBIHBIDIDIHFF------GGFFGFFFGFFFGGFHAGMKG-----EM--GEMF--HMF 528
           ||||||||||||||||||||| .|     ||  ||||  .||...||  |   |   .|          .|   | ...| || || .||           |||||||||||||||||||| .|     ||  ||||  .||     |     |.|       ||.| || |  .| ..|||||||||||||| |    || .|       ||..|.|     .| |.||     ||||||||||||| |    || .|  |    ....|| .|    |  |     |||||||||||||||||      |.|||||||||  | ||| ||.||

In [31]:
print(len(fullseq1), len(fullseq2))
print(len(newseq1), len(newseq2))
print(newseq1)
print('\n')
print(newseq2)

638 638
638 638
HFFGGFFGFFFGFFFGGFHAGMKG-----EM--GEMF--HMFHKFMM--G---HM--LG----------AM---KEBBIDIGH-FHEHFHM----------FFGGFFGFFFGFFFGGFHAGMKG-----EM--GEMF--HMFH----GH----GAMK------MHHGCFF-H--ICHHEFGEGGGFGEGGGFGEG----GGDGEI------HHGCFFH-----IC-HHEF-----GEGGGFGEGGGFGEG----GGDGE--IC---MMMLALAKB----H--F-----GEGGGFGEGGGFGEGGG------FGEGGGFGEGG--G-FGEGGGFFG---FFFGFFFGFFFC--GCFFH-CH-EFGEGGGFGEGGGF-GFFFFM---ALAKBH----F------GEGGGFGEGGGFGEGGG------FGEGGGFGEGG--G-FGEGGGFFG---FFFGFFFGFFFC--GCFFH-CH-EFGEGGGFGEGGGF-GFFFFM--GF-GB--FFGH---FF---FG-G---------G----BI---H-----B--I-F--H---F-H----E-------H---F-HM-----DI-IHFF------GGFFGFFFGFFFGGFHAGMKG-----EM--GEMF--HMF


HFFGGFFGFFFGFFFGGFHAG-HGH----EMF-GEMFG-EMFGGEMMH-GF--H---DGD---------EMH--K-FCHD-GHFFH-FFH-----------FFGGFFGFFFGFFFGGFHAG-HGH----EMF-GEMFG-EMF-----G-----GEM-------MHDG-FFBHD-DC-JKFGEGGGFGEGGGFG-GF---GG-FE-------HHIBFJHA----FCFHEEFH----GEGGGFGEGGGFG-GF---GG-FEH-I----BFJHAL-FBJ---HE-FH----GEGGGFGEGGGFGEGGGE-----FMEGGGFGEGGH-GDFGE-GGJFGE--GFFGF

In [32]:
allthepattern_count = dict(Counter(allthepatterns))
# elimiate the repeated strings

allthepatterns = list(set(allthepatterns))

In [33]:
len(allthepatterns)

63

In [34]:
def count_substringandparents(allthepatterns, allthepattern_count):
    
    issubstring = {}
    hassubstring = {}
    count_as_substring = {}
    count_as_parent = {}

    for i in range(0, len(allthepatterns)-1):
        for j in range(i+1, len(allthepatterns)):
            if allthepatterns[i] in allthepatterns[j]:
                parent = allthepatterns[j]
                kid = allthepatterns[i]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] = 1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1#times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 

            elif allthepatterns[j] in allthepatterns[i]:
                parent = allthepatterns[i]
                kid = allthepatterns[j]
                times = parent.count(kid)

                if kid not in issubstring:
                    issubstring[kid] =1
                    count_as_substring[kid] = times * allthepattern_count[parent]
                else:
                    issubstring[kid] += 1
                    count_as_substring[kid] += times * allthepattern_count[parent]

                if parent not in hassubstring:
                    hassubstring[parent] = 1
                    count_as_parent[parent] = 1 #times
                else:
                    hassubstring[parent] += 1
                    count_as_parent[parent] += 1 #times

    return issubstring, hassubstring, count_as_substring, count_as_parent


In [35]:
issubstring, hassubstring, count_as_substring, count_as_parent = count_substringandparents(allthepatterns, allthepattern_count)
onlyassubstring = set(issubstring) - set(hassubstring)

In [36]:
def decode_strings(string):
    
    decode_dia_intervals = {'A': '6D', 'B': '5D', 'C': '4D', 'D': '3D', 'E': '2D', 'F': '1D', 
                            'G': '1A', 'H': '2A', 'I': '3A', 'J': '4A', 'K': '5A', 'L': '6A', 'M': '0A'}
    
    decode_string = ""

    for letter in string:
        trans = decode_dia_intervals[letter]
        decode_string+=trans

    return decode_string

In [37]:
def printLCSSubStr(X: str, Y: str,
                   m: int, n: int):
 
    # Create a table to store lengths of longest common suffixes of substrings.
    # Note that LCSuff[i][j] contains length of longest common suffix of X[0..i-1] and
    # Y[0..j-1]. The first row and first column entries have no logical meaning,
    # they are used only for simplicity of program
    LCSuff = [[0 for i in range(n + 1)]
                 for j in range(m + 1)]
 
    # To store length of the
    # longest common substring
    length = 0
 
    # To store the index of the cell
    # which contains the maximum value.
    # This cell's index helps in building
    # up the longest common substring
    # from right to left.
    row, col = 0, 0
 
    # Following steps build LCSuff[m+1][n+1]
    # in bottom up fashion.
    for i in range(m + 1):
        for j in range(n + 1):
            if i == 0 or j == 0:
                LCSuff[i][j] = 0
            elif X[i - 1] == Y[j - 1]:
                LCSuff[i][j] = LCSuff[i - 1][j - 1] + 1
                if length < LCSuff[i][j]:
                    length = LCSuff[i][j]
                    row = i
                    col = j
            else:
                LCSuff[i][j] = 0
 
    # if true, then no common substring exists
    if length == 0:
        #print("No common substring")
        return ""
 
    # allocate space for the longest
    # common substring
    resultStr = ['0'] * length
 
    # traverse up diagonally form the
    # (row, col) cell until LCSuff[row][col] != 0
    while LCSuff[row][col] != 0:
        length -= 1
        resultStr[length] = X[row - 1] # or Y[col-1]
 
        # move diagonally up to previous cell
        row -= 1
        col -= 1
 
    # required longest common substring
    return ''.join(resultStr) 


In [16]:
def reduce_pattern_length(seq):
        # the function returns the longest substring that appeared at least once
        # if there is length 4 substring repeated 4 times and a length 7 substring repeated 2 times, we take the length 7
        best_performance = ""
        candidates = []
        for length in range(int(0.3*len(seq)), int(len(seq)*0.5)+1): 
            for start in range(0, len(seq)-length):
                # get all the substrings of this length within the string, save in candidates
                candidates.append(seq[start:start+length])
        count_can = {}
        allthepattern_count = dict(Counter(allthepatterns))

        for candidate in candidates:
                count_can[candidate] = seq.count(candidate)
                if count_can[candidate] > 1:
                    # if it is repeated more than once in the string
                    if len(candidate) > len(best_performance):
                        best_performance = candidate
                    elif len(candidate) == len(best_performance) and count_can[candidate] > count_can[best_performance]:
                        best_performance = candidate
        
        return best_performance

def check_reduction(item):
    success = False
    frequent_substr = reduce_pattern_length(item)
    if len(frequent_substr) > 4 and len(frequent_substr) > 0.3*len(item):
        print("extracted", frequent_substr, "from", item)
        # sucessfully reduced the pattern
        return frequent_substr
    return None

In [17]:
def reduce_length_of_each_pattern(onlyassubstring, patterns_to_reduce):
    beforereduction = {}
    # reducedfinalpat is the new list that contains all valid patterns after reduction plus the ones that does not need to be reduced
    reducedfinalpat = []

    for item in patterns_to_reduce:
        # check the ones that has substring, see if their length can be reduced
        if len(item) > 11  and item not in onlyassubstring:
            #if the extracted pattern is too long, find longest repeated pattern within it
            frequent_substr = check_reduction(item)
            if frequent_substr != None:
                if len(frequent_substr) > 23:
                    # second attempt, if a frequent substring is takend and it is still quite long
                    new_frequent_substr = check_reduction(frequent_substr)
                    if new_frequent_substr != None:
                        # if the second attempt is a success, take the twice reduced pattern
                        reducedfinalpat.append(new_frequent_substr)
                        beforereduction[new_frequent_substr] = item
                    else:
                        # if the second attempt failed, take the result of first reduction
                        reducedfinalpat.append(frequent_substr)
                        beforereduction[frequent_substr] = item
                else:
                    # if the extracted pattern is already not very long
                    reducedfinalpat.append(frequent_substr)
                    beforereduction[frequent_substr] = item
            else:
                # if the reduction failed, take the original
                reducedfinalpat.append(item)
        else:
            # otherwise keep it without reduction
            reducedfinalpat.append(item)
    
    return reducedfinalpat, beforereduction

In [18]:
reducedpat, beforereduction = reduce_length_of_each_pattern(onlyassubstring, allthepatterns)

extracted FGEGGG from EFGEGGGFGEGGGF
extracted GEGGGF from GEGGGFGEGGGFGEGGG


In [584]:
lcscount = {}
dictoflcs = {}
for i in range(0, len(reducedpat)-1):
    for j in range(i+1, len(reducedpat)):
        #print(i, j)
        pat1 = reducedpat[i]
        pat2 = reducedpat[j]
        if len(pat1) > 11 and len(pat2) > 11 and pat1 != pat2:
            lcs = printLCSSubStr(pat1, pat2, len(pat1), len(pat2))
            if len(lcs) >= 4: # this can change... I guess? but better not
                if pat1 not in dictoflcs:
                    dictoflcs[pat1] = []
                if pat2 not in dictoflcs:
                    dictoflcs[pat2] = []
                dictoflcs[pat1].append(lcs)
                dictoflcs[pat2].append(lcs)
                if lcs in lcscount:
                    lcscount[lcs] +=1
                else:
                    lcscount[lcs] = 1

newdict = {}
# print out the common strings that appeared more than 3 times
for temp in lcscount:
    if lcscount[temp] >= 2:
        if temp not in reducedpat:
            print("new pattern:")
        newdict[temp] = lcscount[temp]
        print(temp, decode_strings(temp), lcscount[temp])

# note to self: many of them are substring of themselves, take the longer or shorter?
# take the shorter ones and elimiate the longer ones can be practical but can miss information..
# only for the ones that are not reduceable? only between long strings? does it make sense?

GGFFGFFFGFFFGGFHAG 1A1A1D1D1A1D1D1D1A1D1D1D1A1A1D2A6D1A 2


In [585]:
bestlcss=[]
for pattern in reducedpat:
    #if len(pattern) > 11:
        print("for pattern:", pattern)
        if pattern in beforereduction:
            print("was reduced from", beforereduction[pattern])
        if pattern in dictoflcs:
            baseline = 2
            bestlcs = ""
            for lcs in dictoflcs[pattern]:
                #print(lcs, lcscount[lcs])
                if lcscount[lcs] > baseline and len(lcs) > len(bestlcs):
                    bestlcs = lcs
                    baseline = lcscount[lcs]
            print("best lcs:", bestlcs)
            if bestlcs!= "":
                bestlcss.append(bestlcs)
        else:
            print("no lcs that is longer than 4 for this pattern")

for pattern: GEGGGF
was reduced from GEGGGFGEGGGFG
no lcs that is longer than 4 for this pattern
for pattern: MHHG
no lcs that is longer than 4 for this pattern
for pattern: FFGH
no lcs that is longer than 4 for this pattern
for pattern: HFH
no lcs that is longer than 4 for this pattern
for pattern: FGEGGG
was reduced from EFGEGGGFGEGGGF
no lcs that is longer than 4 for this pattern
for pattern: HMF
no lcs that is longer than 4 for this pattern
for pattern: ALAKBH
no lcs that is longer than 4 for this pattern
for pattern: HFFGGFFGFFFGFFFGGFHAG
best lcs: 
for pattern: HMFHKFMM
no lcs that is longer than 4 for this pattern
for pattern: FFGGFFGFFFGFFFGGFHAG
best lcs: 
for pattern: FGEGGGFGEGG
no lcs that is longer than 4 for this pattern
for pattern: FGEGGG
was reduced from EFGEGGGFGEGGGF
no lcs that is longer than 4 for this pattern
for pattern: IHFF
no lcs that is longer than 4 for this pattern
for pattern: GGFFG
no lcs that is longer than 4 for this pattern
for pattern: HHEF
no lcs tha

In [586]:
lcstaken = list(set(bestlcss))
lcstaken

[]

In [587]:
testpat, count_pattern_test, startonsetstest,  endonsetstest = count_final_pattern(lcstaken, strings, beatstrengths)
testpat

[]

In [588]:
newlist = list(newdict)
testpat, count_pattern_test, startonsetstest,  endonsetstest = count_final_pattern(newlist, strings, beatstrengths)
testpat

Pattern GGFFGFFFGFFFGGFHAG appears in voice no. 1 , first pos starts at  119 end at 137
Pattern GGFFGFFFGFFFGGFHAG appears in voice no. 2 , first pos starts at  68 end at 86


['GGFFGFFFGFFFGGFHAG']

In [453]:
reduced_pattern_count = dict(Counter(reducedpat))

In [454]:
for pattern in reducedpat:
    if pattern in beforereduction:
        # if it was reduced, then it at least appeared more than once in the original string, 
        # which should be extrac counted here
        reduced_pattern_count[pattern] +=1
        if pattern in allthepattern_count:
            # if it was also originally a pattern
            reduced_pattern_count[pattern]+=allthepattern_count[pattern]-1
    else:
        reduced_pattern_count[pattern]+=allthepattern_count[pattern]-1
        # if it was not reduced, let's check if it was repeated at first, we should add them here minus the once that is counted already.

In [455]:
issubstring, hassubstring, count_as_substring, count_as_parent = count_substringandparents(reducedpat, reduced_pattern_count)

In [456]:
# rank the ones that are substrings of another detected pattern
sortis = dict(Counter(issubstring))
sorthas = dict(Counter(hassubstring))
print(len(sorthas))
print(len(sortis))

2
2


In [457]:
# This counts the times each pattern appeared after original segmentation plus the times they are in other patterns,
# plus the times other patterns showed up in them.

def combined_count(sorthas, sortis, count_as_parent, count_as_substring, reduced_pattern_count):
    refined_combined = {}

    for item in reducedpat:
        refined_combined[item] = reduced_pattern_count[item]
        if item in count_as_parent:
            refined_combined[item]+=count_as_parent[item]
        if item in count_as_substring:
            refined_combined[item]+=count_as_substring[item]
    """
    refined_combined = {}

    for item in sorthas:
        if item in sortis:
            # the ones that has substring and also are the substrings
            refined_combined[item] = count_as_parent[item] + count_as_substring[item] + reduced_pattern_count[item]
        else:
            # the ones that has substrings
            refined_combined[item] = count_as_parent[item] + reduced_pattern_count[item]
    # the patterns that are substrings of other patterns but none of the others are substrings of it
    for item in sortis:
        if item not in refined_combined:
            refined_combined[item] = count_as_substring[item] + reduced_pattern_count[item]
        # otherwise is already counted
    """
    return refined_combined



In [458]:
combined = combined_count(sorthas, sortis, count_as_parent, count_as_substring, reduced_pattern_count)

In [459]:
def filter_combined_patterns(combined):
    
    finalpattern = []
    for i in combined: 

        if len(i) < 3:
            # if the length is shorter than 4 notes, discard
            continue

        if combined[i] > 1:
            # keep the ones that has substring or is substring more than once
            finalpattern.append(i)

    finalpattern_combined = {}
        
    for thisone in finalpattern:
        finalpattern_combined[thisone] = combined[thisone]

    dict(Counter(finalpattern_combined))
        
    if len(finalpattern) > 30:
        # if more than 30 patterns, only take the top 30
        finalpattern_combined = dict(sorted(finalpattern_combined.items(), key = lambda x:-x[1], reverse = True)[-20:])
    
    return finalpattern_combined


In [460]:
# filter out the short ones
top20patterns_combined = filter_combined_patterns(combined)

In [461]:
print(dict(Counter(top20patterns_combined)))

{'FMMMIMDI': 3, 'MMMM': 2, 'IMDI': 2, 'MMM': 3}


In [462]:
for printpattern in list(top20patterns_combined):
    print(decode_strings(printpattern))

1D0A0A0A3A0A3D3A
0A0A0A0A
3A0A3D3A
0A0A0A


In [463]:
for printpattern in reducedpat:
    print(decode_strings(printpattern))

3A0A3D3A
1D5A0A5D3A
1D0A0A0A3A0A3D3A
0A0A5D0A
0A0A0A
0A4A0A
0A2D0A
0A0A0A0A
0A1A0A


In [464]:
print("the followings are not ranked well thus discarded in the count")
for i in reducedpat:
    if i not in top20patterns_combined:
        print(decode_strings(i))

the followings are not ranked well thus discarded in the count
1D5A0A5D3A
0A0A5D0A
0A4A0A
0A2D0A
0A1A0A


In [465]:
def find_pattern_pos(string, pattern, beatstrength):
    patpos_start = []
    patpos_end = []
    count = 0
    beatcount = 0
    for m in re.finditer(pattern, string):
        count +=1
        if beatstrength[m.start()] == 1: # if it starts in the beginning of the score, exception
            beatcount += 1
        elif m.start() < 8: # if it starts in the beginning for two times, also count
            beatcount += 0.5
        patpos_start.append(m.start())
        patpos_end.append(m.end())
    return patpos_start, patpos_end, count, beatcount

In [466]:
def count_final_pattern(reducedfinalpat, strings, beatstrengths):

    count_pattern = {}
    refined_finalpat = []
    startonsets = []
    startpitches = []
    endonsets = []
    endpitches = []

    for pattern in reducedfinalpat:
        stringnum = 0
        beat_importance = 0
        for string in strings:
            # find the pattern in eachvoice
            patpos_start, patpos_end, count, beatcount = find_pattern_pos(string, pattern, beatstrengths[stringnum])
            beat_importance += beatcount
            if pattern not in count_pattern:
                count_pattern[pattern] = count
            else:
                count_pattern[pattern] += count
            if count != 0:
                print("Pattern", pattern, "appears in voice no.", stringnum+1, ", first pos starts at ", patpos_start[0], "end at", patpos_end[0])
            num = 0
            for startpos in patpos_start:
                startonsets.append(noteoffsets[stringnum][startpos])
                endonsets.append(noteoffsets[stringnum][patpos_end[num]])
                num+=1
            stringnum+=1
        if count_pattern[pattern] >= 2 and beat_importance >= 1:
            # get rid of the ones that actually appeared less than 2 times in total
            refined_finalpat.append(pattern)
        else:
            if count_pattern[pattern] < 2:
                print("discard pattern", pattern, "because it only appered once in total.")
            elif beat_importance == 0:
                print("discard pattern", pattern, "because it never appeared at the start of a bar.")
                print("this pattern can be decoded as", decode_strings(pattern))
            
    return refined_finalpat, count_pattern, startonsets, endonsets


In [469]:
#finalpat, count_pattern, startonsets,  endonsets = count_final_pattern(reducedfinalpat, strings, beatstrengths)
#
finalpat, count_pattern, startonsets,  endonsets = count_final_pattern(list(top20patterns_combined), strings, beatstrengths)


Pattern FMMMIMDI appears in voice no. 2 , first pos starts at  452 end at 460
discard pattern FMMMIMDI because it only appered once in total.
Pattern MMMM appears in voice no. 1 , first pos starts at  0 end at 4
Pattern MMMM appears in voice no. 2 , first pos starts at  481 end at 485
Pattern IMDI appears in voice no. 2 , first pos starts at  247 end at 251
Pattern MMM appears in voice no. 1 , first pos starts at  0 end at 3
Pattern MMM appears in voice no. 2 , first pos starts at  39 end at 42


In [470]:
def decode_results(finalpat, count_pattern, combined, beforereduction):
    
    decoded_patterns = []
    decoded_patterns_weighed = {}
    decoded_patterns_count = {}
    decoded_patterns_combined = {}
    
    decode_dia_intervals = {'A': '6D', 'B': '5D', 'C': '4D', 'D': '3D', 'E': '2D', 'F': '1D', 
                            'G': '1A', 'H': '2A', 'I': '3A', 'J': '4A', 'K': '5A', 'L': '6A', 'M': '0A'}

    # decode all the candidates for patterns
    for pattern in finalpat:
        trans_pattern = ""
        for letter in pattern:
            trans = decode_dia_intervals[letter]
            trans_pattern += trans

        decoded_patterns_count[trans_pattern] = count_pattern[pattern]
        
        if pattern in combined:
            # some might be reduced to a shorter pattern
            decoded_patterns_combined[trans_pattern] = combined[pattern]
        else:
            decoded_patterns_combined[trans_pattern] = combined[beforereduction[pattern]]
        decoded_patterns.append(trans_pattern)
        
    return decoded_patterns, decoded_patterns_combined, decoded_patterns_count


Strings of intervals of each voice:

In [471]:
# Strings of intervals of each voice
decoded_strings = []
for string in strings:
    decoded = decode_strings(string)
    decoded_strings.append(decoded)
    print(decoded)
    print("")

0A0A0A0A0A6A0A6D0A0A6A0A6D1A0A4A0A4D0A0A4A0A4D1A0A3A3D1A0A0A4A0A4D1A0A3A3D1A0A0A4A0A4D1A0A3A3D1A0A0A0A1A0A1A0A2A1D1D1D4A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A4D1D0A5A2A1D0A6D1A0A1D5D5A4A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A6A6D0A6A6D0A5A6D0A6A1D1A1D1D1D3D0A1D4A1D5D5A5D0A2A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A5A5D0A5A6D0A5A5D0A5A6D0A6A6D0A6A0A3D3A3D3D2A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A4D1D0A5A0A1A0A6D2A1D0A1D1D4A3D3A3D2D1A1D1A1D1D3A1A0A0A1A1D4D1D0A3A6A3D3A3D2D1A1D1A1D1D3A1A0A0A1A1D4D1D0A3A6A3D3A3D2D1A1D1A1D1D3A1A0A4D1D0A5A2A1D0A6D1A0A1D5D5A4A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A6A6D0A6A6D0A5A6D0A6A1D1A1D1D1D3D0A1D4A1D5D5A5D0A2A3D3D2D2A0A4D1D5D5A6A2A0A1D5D5A6D0A6A6D0A6A6D0A5A5D0A5A6D0A6A6D0A5A6D0A5A5D0A5A6D0A5A5D0A5A6D0A6A6D0A6A0A3D3A3D3D2A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D1D3A1A0A0A4D1D0A3A6A3D3A3D2D1A1D

In [472]:
# Decode patterns
decoded_patterns, decoded_patterns_combined, decoded_patterns_count = decode_results(finalpat, count_pattern, combined, beforereduction)

for pattern in decoded_patterns:
        print("Pattern:", pattern)
        print("importance score:", decoded_patterns_combined[pattern],
              "occurrence:", decoded_patterns_count[pattern])


Pattern: 0A0A0A0A
importance score: 2 occurrence: 7
Pattern: 3A0A3D3A
importance score: 2 occurrence: 6
Pattern: 0A0A0A
importance score: 3 occurrence: 19


In [473]:
decoded_patterns

['0A0A0A0A', '3A0A3D3A', '0A0A0A']

In [474]:
# Get patterns from ground truth

dict_wordtonum = {"Unison": '0', "Second": '1', "Third": '2', "Fourth": '3', "Fifth": '4', "Sixth": '5', "Seventh": '6'}

groundtruthpatterns = []
print(glob.glob("jkupddtest/chopinmono/*"))
filenames = glob.glob("jkupddtest/chopinmono/B/*.csv")
for filename in filenames:
    print(filename)
    with open(filename, 'r') as file:
        csvreader = csv.reader(file)
        gts = []
        gtm = []
        pattern_gt = []
        for row in csvreader:
            gts.append(float(row[0]))
            gtm.append(float(row[1]))
        for i in gtm:
            pattern_gt.append(m21.note.Note(i))
        pattern_string = ""
        interval_string = ""
        for i in pattern_gt:
            pattern_string += str(i)
            
        for i in range(1, len(pattern_gt)):           
            gap = pattern_gt[i].pitch.diatonicNoteNum - pattern_gt[i-1].pitch.diatonicNoteNum
            if gap >= 0:
                direction = 'A'
            else:
                direction = 'D'
            m21_interval_directed = m21.interval.Interval(noteStart=pattern_gt[i-1], noteEnd=pattern_gt[i]).directedSimpleNiceName
            arr_diatonic = m21_interval_directed.split(" ")
            m21_generic = dict_wordtonum[arr_diatonic[-1]]
            m21_interval = m21_generic+direction
            
            interval_string+=m21_interval
            
        print(interval_string)

        groundtruthpatterns.append(interval_string)


['jkupddtest/chopinmono/D', 'jkupddtest/chopinmono/A', 'jkupddtest/chopinmono/mazurka24-4.krn']


In [475]:
set(groundtruthpatterns)

set()

In [112]:
for pattern in groundtruthpatterns:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

In [63]:
##CHECK IF THERE IS A MATCH

for i in groundtruthpatterns:
    if i in finalpat: #refined_finalpat:
        print("found a match:", i)

In [64]:
#TEST2


In [65]:
#groundtruthpath1 = "jkupddtest/gibbonsmono/silverswan.krn"
import pandas as pd
gtfiles = glob.glob("jkupddtest/gibbonsmono/H/midi/*.csv")
print(gtfiles)
gtstrings = []
for gtfile in gtfiles:
    print(gtfile)
    with open(gtfile, 'r') as file:
        df = pd.read_csv(file)
        #print(df)
        #for midi_note_num in df["midi_note_num"]:
        #    print(midi_note_num)
        count = 0
        gtstring = ""
        for diatonic in df["diatonic_interval"]:
            if count == 0:
                # ignore the first "interval" number which is always 0
                count+=1
                continue
            count+=1
            # just to format the input
            if diatonic < 0:
                dia = str(abs(diatonic)) + 'D'
            else:
                dia = str(abs(diatonic)) + 'A'
            gtstring += dia
        gtstrings.append(gtstring)
        print(gtstring)
            

['jkupddtest/gibbonsmono/H/midi/occ1.midi_part1_DD.csv', 'jkupddtest/gibbonsmono/H/midi/occ2.midi_part1_DD.csv']
jkupddtest/gibbonsmono/H/midi/occ1.midi_part1_DD.csv
0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D
jkupddtest/gibbonsmono/H/midi/occ2.midi_part1_DD.csv
0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D


In [366]:
print(set(gtstrings))

{'0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D'}


In [367]:
for pattern in gtstrings:
    count = 0
    for decodedstring in decoded_strings:
        count += decodedstring.count(pattern)
    if count == 0:
        print("pattern", pattern, "not in original score")
    elif count == 1:
        print("pattern", pattern, "only appeared once")
    else:
        print("pattern", pattern, "appeared", count, "times in total")

pattern 0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D not in original score
pattern 0A1D1D1D1D0A7A1D1D4D3A1D1A2D1A3D1A0A4D not in original score
